In [1]:
import os
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import numpy as np

In [2]:
# Define the paths
train_dir = './data/train'
valid_dir = './data/valid'
test_dir = './data/test'

# Load the CSV file
dataset_csv = pd.read_csv('./data/cards.csv')

In [3]:
# Create ImageDataGenerator for loading and augmenting images
train_datagen = ImageDataGenerator(rescale=1./255)
valid_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

valid_generator = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

Found 7624 images belonging to 53 classes.
Found 265 images belonging to 53 classes.
Found 265 images belonging to 53 classes.


In [4]:
# Build the model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(53, activation='softmax')  # 53 classes for 53 cards
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

C:\Python\Python3_11_7\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


In [5]:
# Train the model
history = model.fit(
    train_generator,
    epochs=25,
    validation_data=valid_generator
)

# Evaluate the model
loss, accuracy = model.evaluate(test_generator)
print(f'Test accuracy: {accuracy}')

Epoch 1/25


C:\Python\Python3_11_7\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


239/239 ━━━━━━━━━━━━━━━━━━━━ 199s 827ms/step - accuracy: 0.1523 - loss: 3.6165 - val_accuracy: 0.5698 - val_loss: 1.6039
Epoch 2/25
239/239 ━━━━━━━━━━━━━━━━━━━━ 199s 831ms/step - accuracy: 0.5138 - loss: 1.8034 - val_accuracy: 0.7019 - val_loss: 1.0903
Epoch 3/25
239/239 ━━━━━━━━━━━━━━━━━━━━ 197s 825ms/step - accuracy: 0.6946 - loss: 1.0905 - val_accuracy: 0.7887 - val_loss: 0.8715
Epoch 4/25
239/239 ━━━━━━━━━━━━━━━━━━━━ 197s 824ms/step - accuracy: 0.8347 - loss: 0.5975 - val_accuracy: 0.8151 - val_loss: 0.7658
Epoch 5/25
239/239 ━━━━━━━━━━━━━━━━━━━━ 199s 831ms/step - accuracy: 0.9029 - loss: 0.3466 - val_accuracy: 0.7962 - val_loss: 0.8268
Epoch 6/25
239/239 ━━━━━━━━━━━━━━━━━━━━ 197s 823ms/step - accuracy: 0.9273 - loss: 0.2892 - val_accuracy: 0.7962 - val_loss: 0.7954
Epoch 7/25
239/239 ━━━━━━━━━━━━━━━━━━━━ 198s 826ms/step - accuracy: 0.9539 - loss: 0.1688 - val_accuracy: 0.8264 - val_loss: 0.7537
Epoch 8/25
239/239 ━━━━━━━━━━━━━━━━━━━━ 198s 830ms/step - accuracy: 0.9624 - loss: 0.14

In [ ]:
with open('model.pkl', 'wb') as file:
    pickle.dump(model, file)

In [ ]:
with open('model.pkl', 'rb') as file:
    loaded_model = pickle.load(file)

In [10]:
# Define card counting function
def classify_card(image):
    image = tf.image.resize(image, (224, 224))
    image = tf.expand_dims(image, 0)  # Add batch dimension
    predictions = model.predict(image)
    class_index = tf.argmax(predictions[0]).numpy()
    class_label = list(train_generator.class_indices.keys())[class_index]
    return class_label

def count_cards(images):
    card_counts = {label: 0 for label in train_generator.class_indices.keys()}
    print(card_counts)
    global_count = 0
    high_cards = {'10', 'J', 'Q', 'K', 'A'}
    low_cards = {'2', '3', '4', '5', '6'}
    total_cards = 52 * 8  # Total number of cards in a standard deck

    for image in images:
        card_label = classify_card(image)
        card_counts[card_label] += 1
        if card_label in high_cards:
            global_count -= 1
        elif card_label in low_cards:
            global_count += 1

    # Calculate remaining cards
    remaining_cards = total_cards - sum(card_counts.values())

    # Calculate probabilities
    remaining_high_cards = sum(str(card_counts[card]) for card in high_cards)
    remaining_low_cards = sum(str(card_counts[card]) for card in low_cards)
    remaining_neutral_cards = remaining_cards - remaining_high_cards - remaining_low_cards

    prob_high = remaining_high_cards / remaining_cards if remaining_cards > 0 else 0
    prob_low = remaining_low_cards / remaining_cards if remaining_cards > 0 else 0
    prob_neutral = remaining_neutral_cards / remaining_cards if remaining_cards > 0 else 0

    return card_counts, global_count, prob_high, prob_low, prob_neutral

In [11]:
# Example usage with a list of images
image_paths = [r'C:\Users\Amram\IMPORTANT\Projects\github_projects\BlackJackAI\data\train\ace of clubs\003.jpg', r'C:\Users\Amram\IMPORTANT\Projects\github_projects\BlackJackAI\data\train\ace of clubs\004.jpg']
images = [tf.io.read_file(image_path) for image_path in image_paths]
images = [tf.image.decode_jpeg(image, channels=3) for image in images]

card_counts, global_count, prob_high, prob_low, prob_neutral = count_cards(images)
print(f'Card counts: {card_counts}')
print(f'Global count: {global_count}')
print(f'Probability of next card being +1 count: {prob_low:.2f}')
print(f'Probability of next card being -1 count: {prob_high:.2f}')
print(f'Probability of next card being 0 count: {prob_neutral:.2f}')

{'ace of clubs': 0, 'ace of diamonds': 0, 'ace of hearts': 0, 'ace of spades': 0, 'eight of clubs': 0, 'eight of diamonds': 0, 'eight of hearts': 0, 'eight of spades': 0, 'five of clubs': 0, 'five of diamonds': 0, 'five of hearts': 0, 'five of spades': 0, 'four of clubs': 0, 'four of diamonds': 0, 'four of hearts': 0, 'four of spades': 0, 'jack of clubs': 0, 'jack of diamonds': 0, 'jack of hearts': 0, 'jack of spades': 0, 'joker': 0, 'king of clubs': 0, 'king of diamonds': 0, 'king of hearts': 0, 'king of spades': 0, 'nine of clubs': 0, 'nine of diamonds': 0, 'nine of hearts': 0, 'nine of spades': 0, 'queen of clubs': 0, 'queen of diamonds': 0, 'queen of hearts': 0, 'queen of spades': 0, 'seven of clubs': 0, 'seven of diamonds': 0, 'seven of hearts': 0, 'seven of spades': 0, 'six of clubs': 0, 'six of diamonds': 0, 'six of hearts': 0, 'six of spades': 0, 'ten of clubs': 0, 'ten of diamonds': 0, 'ten of hearts': 0, 'ten of spades': 0, 'three of clubs': 0, 'three of diamonds': 0, 'three 

KeyError: '10'